### Extracting, transforming, and selecting features (for class)

In this notebook we demonstrate some of the functionality available in Spark MLlib to extract, transform, and select features for machine learning.  In particular, we generate features from the ride review text that can be used to predict the ride rating.  We cover a small subset of the available transformations here but will cover additional transformations in subsequent modules. This notebook is based on material supplied by Cloudera under their Cloudera Academic Partner program and *Spark: The Definitive Guide* book by Bill Chambers and Matei Zaharia. 

Topics: Extracting and transforming features
- Tokenizer
- RegexTokenizer
- StopWordsRemover
- CountVectorizer (convert documents of words to numbers)
- ChiSqSelector

You can find details of all of these *Transformers*, *Estimators* along with their methods and attributes in the [Spark MLlib API Reference](https://spark.apache.org/docs/latest/api/python/reference/pyspark.ml.html) and a more general guide to their use in the [Machine Learning Library (MLlib) Guide](https://spark.apache.org/docs/latest/ml-guide.html)

In [0]:
# Read the ride review data
reviews = spark.read.parquet("/mnt/cis442f-data/duocar/clean/ride_reviews/")

# Read the ride data
rides = spark.read.parquet("/mnt/cis442f-data/duocar/clean/rides/")

# We only want the rides with reviews, so left outer join `reviews` and `rides`
joined = reviews.join(rides, reviews.ride_id == rides.id, "left_outer")

# Select the subset of columns in which we are interested
reviews_with_ratings = joined.select("ride_id", "review", "star_rating")

# This small sample of the resulting DataFrame indicates that there may well
# be a relationship between the review and the rating (eyeball algorithm)
for item in reviews_with_ratings.head(5):
    print (item)    

Row(ride_id='0000000009', review='Dale is extremely cordial.', star_rating=5)
Row(ride_id='0000000037', review='Very junky car.', star_rating=2)
Row(ride_id='0000000071', review='most awful stench of all time! throw away your air freshener!', star_rating=2)
Row(ride_id='0000000083', review='No trouble of note.', star_rating=3)
Row(ride_id='0000000086', review='The driver drove so well!!', star_rating=5)

### Extracting and transforming features

The ride reviews are not in a form amenable to machine learning algorithms. Spark MLlib provides a number of feature extractors and feature transformers to preprocess the ride reviews into a form appropriate for modeling. We start with _tokenizing_ the reviews data using `Tokenizer`.

**Note:** 
- `Tokenizer` is a *Transformer* since it takes a DataFrame as input and returns a DataFrame as output via its `transform` method.
- The Data Type returned is an *arrary*, one of Spark's complex types
- Punctuation is not being handled properly.  I will ask you to use the `RegexTokenizer` class to split on characters other than whitespace for one of the hands on exercises.

In [0]:
# Use the 'Tokenizer' class to tokenize the reviews
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="review", outputCol="words")
tokenized = tokenizer.transform(reviews_with_ratings)

# tokenized.show(5,False)

In [0]:
# Examine a sample of tokens
for item in tokenized.select("review", "words").head(5):
    print (item[0], item[1])
    # print ("\n")

Dale is extremely cordial. ['dale', 'is', 'extremely', 'cordial.']
Very junky car. ['very', 'junky', 'car.']
most awful stench of all time! throw away your air freshener! ['most', 'awful', 'stench', 'of', 'all', 'time!', 'throw', 'away', 'your', 'air', 'freshener!']
No trouble of note. ['no', 'trouble', 'of', 'note.']
The driver drove so well!! ['the', 'driver', 'drove', 'so', 'well!!']

`CountVectorizer` and `CountVectorizerModel` aim to help convert a collection of text documents (the rows of the `words` column in our case) to vectors of token counts. When an a-priori dictionary is not available, `CountVectorizer` can be used as an Estimator to extract the vocabulary, and generates a `CountVectorizerModel`. The model produces sparse representations for the documents over the vocabulary, which can then be passed to other algorithms.

During the fitting process, `CountVectorizer` will select the top `vocabSize` words ordered by term frequency across the corpus. An optional parameter `minDF` also affects the fitting process by specifying the minimum number (or fraction if < 1.0) of documents a term must appear in to be included in the vocabulary. Another optional `binary` toggle parameter controls the output vector. If set to true all nonzero counts are set to 1. This is especially useful for discrete probabilistic models that model binary, rather than integer, counts. [Source of this description](https://spark.apache.org/docs/latest/ml-features.html#countvectorizer)

In [0]:
# Next we use the `CountVectorizer` class to compute the term frequency
# Will produce top ten most frequent words are a vocabulary

from pyspark.ml.feature import CountVectorizer
vectorizer = CountVectorizer(inputCol="words", outputCol="words_vectorized", vocabSize=10)
vectorizer_model = vectorizer.fit(tokenized)
vectorized = vectorizer_model.transform(tokenized)
# vectorized.select("words", "words_vectorized").head(5)

In [0]:
# Examine vectors
for item in vectorized.select("words", "words_vectorized").head(5):
    print (item[0], item[1])

['dale', 'is', 'extremely', 'cordial.'] (10,[],[])
['very', 'junky', 'car.'] (10,[],[])
['most', 'awful', 'stench', 'of', 'all', 'time!', 'throw', 'away', 'your', 'air', 'freshener!'] (10,[5],[1.0])
['no', 'trouble', 'of', 'note.'] (10,[],[])
['the', 'driver', 'drove', 'so', 'well!!'] (10,[0,2],[1.0,1.0])

**Note:**
- `CountVectorizer` is an `Estimator` since it takes a DataFrame as input and returns a `Transformer` as output via its `fit` method.  In this case, the resulting transformer is an instance of the  `CountVectorizerModel` class.
- The resulting word vector is stored in sparse format. First number is the `vocabSize`, the dictionary key is the index of the word in the `vocabulary` (see below) and the dictionary value is the frequency of the word in the document (the review data for a ride in this example)
- Our limited vocabulary (see below) includes a number of common words such as "the" that we do not expect to be predictive


The [HashingTF](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.HashingTF.html#pyspark.ml.feature.HashingTF) class is an alternative for computing the term frequency which we can try in the Hands On exercises

In [0]:
# Examine vocabulary identified by CountVectorizer
list(enumerate(vectorizer_model.vocabulary)) 

Out[7]: [(0, 'the'),
 (1, 'to'),
 (2, 'driver'),
 (3, 'ride'),
 (4, 'was'),
 (5, 'air'),
 (6, 'i'),
 (7, 'and'),
 (8, 'a'),
 (9, 'this')]

Spark MLlib provides a transformer to remove these so-called "stop words".  Use the `StopWordsRemover`
 class to remove common words

In [0]:
# Use the `StopWordsRemover` class to remove common words

from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="words_removed")
removed = remover.transform(tokenized)

# Show Stopwords
remover.getStopWords() 

Out[8]: ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 'should',
 'now',
 "i'll",
 "you'll",
 "he'll",
 "she'll",
 "we'll",
 "they'll",
 "i'd",
 "you'd",
 "he'd",
 "she'd",
 "we'd",
 "they'd",
 "i'm",
 "you're",
 "he's",
 "she's",
 "it's",
 "we're",
 "they're",
 "i've",
 "we've",
 "you've",
 "they've",
 "isn't",
 "aren't",
 "wasn't",
 "weren't",
 "haven't",
 "hasn't",
 "hadn't",
 "don't",
 "doesn't",
 "didn't",
 "won't",
 "wouldn't",
 "shan't",
 "shouldn't",
 "mustn't",
 "can't",
 "couldn't",
 'cannot',
 'could',
 "here's",
 "how's",
 "let's",
 'ought',
 "that's",
 "there's",
 "what's",
 "when's",
 "where's",
 "who's",
 "why's",
 'would']

In [0]:
# Compare tokenized text and new column without stop words
for item in removed.select("words", "words_removed").head(5):
    print (item[0], item[1]) 

['dale', 'is', 'extremely', 'cordial.'] ['dale', 'extremely', 'cordial.']
['very', 'junky', 'car.'] ['junky', 'car.']
['most', 'awful', 'stench', 'of', 'all', 'time!', 'throw', 'away', 'your', 'air', 'freshener!'] ['awful', 'stench', 'time!', 'throw', 'away', 'air', 'freshener!']
['no', 'trouble', 'of', 'note.'] ['trouble', 'note.']
['the', 'driver', 'drove', 'so', 'well!!'] ['driver', 'drove', 'well!!']

In [0]:
# Recount the words
vectorizer = CountVectorizer(inputCol="words_removed", outputCol="words_vectorized", vocabSize=10)
vectorizer_model = vectorizer.fit(removed)
vectorized = vectorizer_model.transform(removed)
vectorized.select("words_removed", "words_vectorized").head(5)

# **Note:** Our vocabulary seems more reasonable now. Although the problems with punctuation remain
list(enumerate(vectorizer_model.vocabulary))

Out[10]: [(0, 'driver'),
 (1, 'ride'),
 (2, 'air'),
 (3, 'ride.'),
 (4, 'car'),
 (5, 'really'),
 (6, 'freshener'),
 (7, 'vehicle'),
 (8, 'due'),
 (9, 'freshener.')]

#### Selecting features

We have generated a potentially large number of features.  How do we distinguish the relevant features from the irrelevant ones?  Spark MLlib provides the [ChiSqSelector](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.ChiSqSelector.html#pyspark.ml.feature.ChiSqSelector) estimator to address this challenge when the label is categorical.

In [0]:
# Use `ChiSqSelector` to select the top 5 features (probabaly too few to work well)
from pyspark.ml.feature import ChiSqSelector
selector = ChiSqSelector(featuresCol="words_vectorized", labelCol="star_rating", outputCol="words_selected", numTopFeatures=5)
selector_model = selector.fit(vectorized)
selected = selector_model.transform(vectorized)

# View reduced 
for item in selected.select("words_removed", "words_selected").head(5):
    print (item[0], item[1])
 

['dale', 'extremely', 'cordial.'] (5,[],[])
['junky', 'car.'] (5,[],[])
['awful', 'stench', 'time!', 'throw', 'away', 'air', 'freshener!'] (5,[1],[1.0])
['trouble', 'note.'] (5,[],[])
['driver', 'drove', 'well!!'] (5,[],[])

In [0]:
# View reduced 
for item in selected.select("words_removed", "words_selected").head(50):
    print (item[0], item[1])

['dale', 'extremely', 'cordial.'] (5,[],[])
['junky', 'car.'] (5,[],[])
['awful', 'stench', 'time!', 'throw', 'away', 'air', 'freshener!'] (5,[1],[1.0])
['trouble', 'note.'] (5,[],[])
['driver', 'drove', 'well!!'] (5,[],[])
['really', 'irritating', 'driver', 'refused', 'cease', 'chatting!!'] (5,[],[])
['breathe', 'due', 'air', 'freshener.'] (5,[1,4],[1.0,1.0])
['adequate'] (5,[],[])
['driver', 'drove', 'nicely!!'] (5,[],[])
['pleased', 'ride.'] (5,[2],[1.0])
['stereo', 'blaring'] (5,[],[])
['made', 'expected'] (5,[],[])
['driver', 'polite', 'wanted', 'talk'] (5,[],[])
['delightful', 'ride'] (5,[0],[1.0])
['driver', 'drove', 'super', 'well.'] (5,[],[])
['car', 'stunk!', 'duo', 'car', 'really', 'ban', 'air', 'fresheners', 'fragrances'] (5,[1],[1.0])
['lovely', 'ride.'] (5,[2],[1.0])
['fully', 'pleased'] (5,[],[])
['messy', 'car!', 'vacuum', 'mess.'] (5,[],[])
['gross', 'ride', 'due', 'really', 'reeking', 'air', 'freshener!'] (5,[0,1,4],[1.0,1.0,1.0])
['marvelous', 'ride', 'friendly', 'driver'] (5,[0],[1.0])
['breathe', 'air', 'freshener.'] (5,[1],[1.0])
['polite', 'driver!', 'refused', 'roll', 'windows', 'even', 'asked', 'to!'] (5,[],[])
['pleasing', 'ride'] (5,[0],[1.0])
['adequate', 'driver.'] (5,[],[])
['satisfactory', 'driver'] (5,[],[])
['completely', 'happy', 'ride.'] (5,[2],[1.0])
['air', 'freshener', 'vehicle', 'smelled'] (5,[1,3],[1.0,1.0])
['rough', 'ride!!!', 'driver', 'taught', 'drive!!!'] (5,[],[])
['michael', 'cease', 'chatting!', 'spoke', 'on.', 'want', 'chat.'] (5,[],[])
['superb', 'time', 'really', 'courteous', 'driver'] (5,[],[])
['thanks', 'ride.'] (5,[2],[1.0])
['happy', 'ride.'] (5,[2],[1.0])
['nasty', 'ride', 'due', 'reeking', 'air', 'freshener.'] (5,[0,1,4],[1.0,1.0,1.0])
['driver', 'talk'] (5,[],[])
['pleased', 'ride', 'quite', 'fine', 'time'] (5,[0],[1.0])
['exceptional', 'service.'] (5,[],[])
['happy', 'ride', 'nice', 'time'] (5,[0],[1.0])
['perfectly', 'pleased!'] (5,[],[])
['ride', 'ok'] (5,[0],[1.0])
['totally', 'ok', 'time'] (5,[],[])
['pleasing', 'ride.'] (5,[2],[1.0])
['music', 'noisy', 'hear', 'driver', 'saying'] (5,[],[])
['awful', 'stink', 'world!', 'throw', 'away', 'air', 'freshener.'] (5,[1],[1.0])
['jeremiah', 'drives', 'super', 'nicely!!'] (5,[],[])
['ride', 'decent', 'enough.'] (5,[0],[1.0])
['driver', 'attempted', 'speak'] (5,[],[])
['worst', 'odor', 'ever!', 'pitch', 'air', 'freshener!'] (5,[1],[1.0])
['karli', 'drives', 'super', 'well!!!'] (5,[],[])
['pleased', 'ride.'] (5,[2],[1.0])

In [0]:
# List selected words:
[vectorizer_model.vocabulary[i] for i in selector_model.selectedFeatures]

Out[13]: ['ride', 'air', 'ride.', 'freshener', 'due']

In [0]:
# `ChiSqSelector` does not provide any information on relative strength or
# direction of the predictive relationships.  We can use some Spark SQL
# to compute the average ride rating based on the presence or absence of each
# vocabulary word in a review

from pyspark.sql.functions import array_contains, count, mean
for word in vectorizer_model.vocabulary:
  print ("**** word = %s ****\n" % word)
  vectorized \
    .select(array_contains("words_removed", word).alias("contains_word"), "star_rating") \
    .groupBy("contains_word") \
    .agg(count("star_rating"), mean("star_rating")) \
    .show()

# These results appear to be consistent with the results of `ChiSqSelector`


**** word = driver ****

+-------------+------------------+------------------+
contains_word|count(star_rating)| avg(star_rating)|
+-------------+------------------+------------------+
 true| 477|2.6624737945492662|
 false| 1345|3.0438661710037174|
+-------------+------------------+------------------+

**** word = ride ****

+-------------+------------------+------------------+
contains_word|count(star_rating)| avg(star_rating)|
+-------------+------------------+------------------+
 true| 380|2.7342105263157896|
 false| 1442|2.9993065187239942|
+-------------+------------------+------------------+

**** word = air ****

+-------------+------------------+------------------+
contains_word|count(star_rating)| avg(star_rating)|
+-------------+------------------+------------------+
 true| 321|1.5950155763239875|
 false| 1501|3.2325116588940705|
+-------------+------------------+------------------+

**** word = ride. ****

+-------------+------------------+------------------+
contains_word|count(star_rating)| avg(star_rating)|
+-------------+------------------+------------------+
 true| 168|3.8511904761904763|
 false| 1654| 2.851874244256348|
+-------------+------------------+------------------+

**** word = car ****

+-------------+------------------+------------------+
contains_word|count(star_rating)| avg(star_rating)|
+-------------+------------------+------------------+
 true| 158|2.4430379746835444|
 false| 1664|2.9915865384615383|
+-------------+------------------+------------------+

**** word = really ****

+-------------+------------------+------------------+
contains_word|count(star_rating)| avg(star_rating)|
+-------------+------------------+------------------+
 true| 147|2.9863945578231292|
 false| 1675|2.9402985074626864|
+-------------+------------------+------------------+

**** word = freshener ****

+-------------+------------------+------------------+
contains_word|count(star_rating)| avg(star_rating)|
+-------------+------------------+------------------+
 true| 140|1.5785714285714285|
 false| 1682| 3.057669441141498|
+-------------+------------------+------------------+

**** word = vehicle ****

+-------------+------------------+------------------+
contains_word|count(star_rating)| avg(star_rating)|
+-------------+------------------+------------------+
 true| 129|2.4031007751937983|
 false| 1693| 2.985233313644418|
+-------------+------------------+------------------+

**** word = due ****

+-------------+------------------+------------------+
contains_word|count(star_rating)| avg(star_rating)|
+-------------+------------------+------------------+
 true| 114|1.5964912280701755|
 false| 1708|3.0339578454332554|
+-------------+------------------+------------------+

**** word = freshener. ****

+-------------+------------------+------------------+
contains_word|count(star_rating)| avg(star_rating)|
+-------------+------------------+------------------+
 true| 80| 1.675|
 false| 1742|3.0022962112514353|
+-------------+------------------+------------------+

In [0]:
Copied results of previous cell to make all of them visible 
**** word = driver ****
+-------------+------------------+------------------+
|contains_word|count(star_rating)|  avg(star_rating)|
+-------------+------------------+------------------+
|         true|               477|2.6624737945492662|
|        false|              1345|3.0438661710037174|
+-------------+------------------+------------------+

**** word = ride ****
+-------------+------------------+------------------+
|contains_word|count(star_rating)|  avg(star_rating)|
+-------------+------------------+------------------+
|         true|               380|2.7342105263157896|
|        false|              1442|2.9993065187239942|
+-------------+------------------+------------------+

**** word = air ****
+-------------+------------------+------------------+
|contains_word|count(star_rating)|  avg(star_rating)|
+-------------+------------------+------------------+
|         true|               321|1.5950155763239875|
|        false|              1501|3.2325116588940705|
+-------------+------------------+------------------+

**** word = ride. ****
+-------------+------------------+------------------+
|contains_word|count(star_rating)|  avg(star_rating)|
+-------------+------------------+------------------+
|         true|               168|3.8511904761904763|
|        false|              1654| 2.851874244256348|
+-------------+------------------+------------------+

**** word = car ****
+-------------+------------------+------------------+
|contains_word|count(star_rating)|  avg(star_rating)|
+-------------+------------------+------------------+
|         true|               158|2.4430379746835444|
|        false|              1664|2.9915865384615383|
+-------------+------------------+------------------+

**** word = really ****
+-------------+------------------+------------------+
|contains_word|count(star_rating)|  avg(star_rating)|
+-------------+------------------+------------------+
|         true|               147|2.9863945578231292|
|        false|              1675|2.9402985074626864|
+-------------+------------------+------------------+

**** word = freshener ****
+-------------+------------------+------------------+
|contains_word|count(star_rating)|  avg(star_rating)|
+-------------+------------------+------------------+
|         true|               140|1.5785714285714285|
|        false|              1682| 3.057669441141498|
+-------------+------------------+------------------+

**** word = vehicle ****
+-------------+------------------+------------------+
|contains_word|count(star_rating)|  avg(star_rating)|
+-------------+------------------+------------------+
|         true|               129|2.4031007751937983|
|        false|              1693| 2.985233313644418|
+-------------+------------------+------------------+

**** word = due ****
+-------------+------------------+------------------+
|contains_word|count(star_rating)|  avg(star_rating)|
+-------------+------------------+------------------+
|         true|               114|1.5964912280701755|
|        false|              1708|3.0339578454332554|
+-------------+------------------+------------------+

**** word = freshener. ****
+-------------+------------------+------------------+
|contains_word|count(star_rating)|  avg(star_rating)|
+-------------+------------------+------------------+
|         true|                80|             1.675|
|        false|              1742|3.0022962112514353|
+-------------+------------------+------------------+


File "<command-3231183002850899>" , line 1 
 Copied results of previous cell to make all of them visible 
 ^
 SyntaxError : invalid syntax

#### Predict ride ratings

As a preview of topics to come, let us build a Naive Bayes classifier to predict the ride rating from the word vector.  Before doing so, we need to preprocess the ride rating.

Classification algorithms in Spark MLlib generally assume that the label is a zero-based integer.  Let us subtract one from the original star_rating to comply:

In [0]:
from pyspark.sql.functions import col
indexed = selected.withColumn("star_rating_indexed", col("star_rating") - 1.0)


In [0]:
# We can examine the mapping by applying the `crosstab` method:
indexed \
  .crosstab("star_rating", "star_rating_indexed") \
  .orderBy("star_rating_star_rating_indexed") \
  .show()

+-------------------------------+---+---+---+---+---+
star_rating_star_rating_indexed|0.0|1.0|2.0|3.0|4.0|
+-------------------------------+---+---+---+---+---+
 1|419| 0| 0| 0| 0|
 2| 0|469| 0| 0| 0|
 3| 0| 0|320| 0| 0|
 4| 0| 0| 0| 23| 0|
 5| 0| 0| 0| 0|591|
+-------------------------------+---+---+---+---+---+

In [0]:
# Now we are ready to build a simple Naive Bayes classifier:
from pyspark.ml.classification import NaiveBayes
naive_bayes = NaiveBayes(featuresCol="words_selected", labelCol="star_rating_indexed")
reviews_with_prediction = naive_bayes.fit(indexed).transform(indexed)

In [0]:
# Compute the *confusion matrix* via the `crosstab` method:
reviews_with_prediction \
  .crosstab("prediction", "star_rating_indexed") \
  .orderBy("prediction_star_rating_indexed") \
  .show()

+------------------------------+---+---+---+---+---+
prediction_star_rating_indexed|0.0|1.0|2.0|3.0|4.0|
+------------------------------+---+---+---+---+---+
 0.0| 60| 44| 9| 0| 0|
 1.0|102|103| 15| 1| 0|
 4.0|257|322|296| 22|591|
+------------------------------+---+---+---+---+---+

In [0]:
# Compute the accuracy of the model:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", \
                                              labelCol="star_rating_indexed", \
                                              metricName="accuracy")
evaluator.evaluate(reviews_with_prediction)

# It looks like we have some more work to do to improve our model.
# Five words will not suffice!
# This model is only correct 41% of the time

Out[20]: 0.4138309549945115

###Hands On

![Hands-on](https://cis442f-open-data.s3.amazonaws.com/pictures/hands.png "Hands-on")


I suggest you clone this notebook to work on these exercises. Then modify the code above as needed. 

#### Exercises

(1) Use the `RegexTokenizer` transformer to more cleanly tokenize the reviews.

(2) Determine if increasing the vocabulary size improves the solution.

(3) Use the `HashingTF` estimator rather than the `CountVectorizer` estimator to generate the term-frequency vectors. `HashingTF` is a more efficient alternative to CountVectorizer (see discussion in references)

We could also seek to improve the model by
- Using ngrams
- Tuning of hyperparameters, training parameters, or prediction parameters
- Trying other algorithms



#### References

[Good discussion about differences between the `HashingTF` and the `CountVectorizer` estimators](https://stackoverflow.com/questions/35205865/what-is-the-difference-between-hashingtf-and-countvectorizer-in-spark)

[Feature engineering](https://en.wikipedia.org/wiki/Feature_engineering)

[Feature selection](https://en.wikipedia.org/wiki/Feature_selection)

[Extracting, transforming, and selecting features](http://spark.apache.org/docs/latest/ml-features.html)